# Lightweight Fine-Tuning Project

* PEFT technique: [`LoRA`](https://huggingface.co/docs/peft/conceptual_guides/adapter#low-rank-adaptation-lora)
* Model: [`gpt2`](https://huggingface.co/openai-community/gpt2)
* Evaluation approach: Hugging Face's [`Trainer.evaluate`](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Trainer.evaluate)
* Fine-tuning dataset: [`imdb`](https://huggingface.co/datasets/imdb)

In [ ]:
import torch

from datasets import load_dataset
from IPython.display import display_markdown, Markdown

## Choosing PyTorch's device based on available backends

In [ ]:
pytorch_device = torch.device("cpu")

In [ ]:
if torch.cuda.is_available():
    pytorch_device = torch.device("cuda")
    display_markdown(Markdown('<div class="alert alert-block alert-success">Using CUDA backend</div>'))
elif torch.backends.mps.is_available():
    pytorch_device = torch.device("mps")
    display_markdown(Markdown('<div class="alert alert-block alert-success">Using MPS backend</div>'))
else:
    display_markdown(Markdown('<div class="alert alert-block alert-warning">Using CPU backend</div>'))

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

### Load dataset

In [ ]:
train_dataset, test_dataset = load_dataset("imdb", split=("train", "test"))

In [ ]:
train_dataset.to_pandas().sample(5)

In [ ]:
test_dataset.to_pandas().sample(5)

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.